In [ ]:
!git clone -b master  https://github.com/neuraloperator/neuraloperator.git

Cloning into 'neuraloperator'...
remote: Enumerating objects: 8320, done.
remote: Counting objects: 100% (2216/2216), done.
remote: Compressing objects: 100% (819/819), done.
remote: Total 8320 (delta 1433), reused 2098 (delta 1371), pack-reused 6104
Receiving objects: 100% (8320/8320), 77.94 MiB | 32.13 MiB/s, done.
Resolving deltas: 100% (5416/5416), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/Darcy_241" -d "/content/drive/My Drive/darcy flow"


Archive:  /content/drive/My Drive/Darcy_241.zip
  inflating: /content/drive/My Drive/darcy flow/piececonst_r241_N1024_smooth2.mat  
  inflating: /content/drive/My Drive/darcy flow/piececonst_r241_N1024_smooth1.mat  


In [ ]:
import scipy.io


mat_contents = scipy.io.loadmat('/content/drive/My Drive/darcy flow/piececonst_r241_N1024_smooth2.mat')

print(mat_contents.keys())

dict_keys(['__header__', '__version__', '__globals__', 'Kcoeff', 'Kcoeff_x', 'Kcoeff_y', 'coeff', 'sol'])


In [ ]:
a = mat_contents['coeff']
u = mat_contents['sol']
print(a.shape)

print(u.shape)

(1024, 241, 241)
(1024, 241, 241)


In [ ]:
%cd /content/neuraloperator/

!python fourier_2d.py

/content/neuraloperator
2376449
0 2.8796648709999317 0.17721805167198182 0.09817358374595642
1 1.5055426809999517 0.08347031199932098 0.07191134095191956
2 1.4919887089999975 0.07179096043109894 0.06768074750900269
3 1.4821427050000011 0.06619802725315094 0.06412598490715027
4 1.483714630999998 0.05958167457580566 0.05437434077262879
5 1.4879619620000994 0.04957132267951965 0.041406763195991514
6 1.61178402500002 0.03979784840345383 0.0328026294708252
7 1.558758881000017 0.03156150126457214 0.029121206402778627
8 1.5423857229999385 0.027838794738054274 0.025332280695438386
9 1.549646359999997 0.026820313036441803 0.02439480632543564
10 1.5236153280000053 0.02443039408326149 0.020247618854045867
11 1.4935071680000647 0.022482516705989837 0.02757486581802368
12 1.4894253889999618 0.02040376764535904 0.018794909715652466
13 1.4894396689999212 0.018686255663633345 0.01790377140045166
14 1.4894517540000152 0.017438278257846832 0.019417176246643065
15 1.494964280999966 0.016094275146722795 0

In [ ]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable

class autoEncoder(nn.Module):
    def __init__(self):
        super(autoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(3364, 1024),
            #nn.Linear(256,128),
            nn.ReLU(True),
            #nn.Linear(128, 64),
            #nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3)
            )

    def forward(self, x):
        x = self.encoder(x)
        #x = self.decoder(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
      super().__init__()
      self.decoder = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(True),
            #nn.Linear(12, 64),
            #nn.ReLU(True),
            #nn.Linear(64, 128),
            #nn.ReLU(True),
            nn.Linear(2048, 3364), nn.Tanh())

    def forward(self,x):
      x = self.decoder(x)
      return x


In [ ]:
"""
@author: Zongyi Li
This file is the Fourier Neural Operator for 2D problem such as the Darcy Flow discussed in Section 5.2 in the [paper](https://arxiv.org/pdf/2010.08895.pdf).
"""

import torch.nn.functional as F
from timeit import default_timer
from utilities3 import *

torch.manual_seed(0)
np.random.seed(0)

class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  #Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        #print('conv1d:',x.shape)
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1,  device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)

        #Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class MLP(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels):
        super(MLP, self).__init__()
        self.mlp1 = nn.Conv1d(in_channels, mid_channels, 1)
        self.mlp2 = nn.Conv1d(mid_channels, out_channels, 1)

    def forward(self, x):
        #print('mlp ',x.shape)
        x = self.mlp1(x)
        x = F.gelu(x)
        x = self.mlp2(x)
        return x


class FNO2d(nn.Module):
    def __init__(self, modes1, modes2,  width):
        super(FNO2d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .

        input: the solution of the coefficient function and locations (a(x, y), x, y)
        input shape: (batchsize, x=s, y=s, c=3)
        output: the solution
        output shape: (batchsize, x=s, y=s, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.width = width
        self.padding = 9 # pad the domain if input is non-periodic

        self.p = nn.Linear(3, self.width) # input channel is 3: (a(x, y), x, y)
        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.mlp0 = MLP(self.width, self.width, self.width)
        self.mlp1 = MLP(self.width, self.width, self.width)
        self.mlp2 = MLP(self.width, self.width, self.width)
        self.mlp3 = MLP(self.width, self.width, self.width)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)
        self.q = MLP(self.width, 1, self.width * 4) # output channel is 1: u(x, y)
        self.encoder = autoEncoder()
        self.decoder = Decoder()

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.p(x)
        x = x.permute(0, 3, 1, 2)
        #print(x.shape) 20, 49,49,30
        #now we do an encoder
        x = F.pad(x, [0,self.padding, 0,self.padding])
        x = x.reshape(20,self.width,-1)
        #print(x.shape) #20 32 2264
        x = self.encoder(x)
        #print(x.shape) #20 32 1024


        x1 = self.conv0(x)
        x1 = self.mlp0(x1)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv1(x)
        x1 = self.mlp1(x1)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv2(x)
        x1 = self.mlp2(x1)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv3(x)
        x1 = self.mlp3(x1)
        x2 = self.w3(x)
        x = x1 + x2
        #print('after 3 mlp: ', x.shape)

        # x = x[..., :-self.padding] # pad the domain if input is non-periodic
        #x = self.q(x)
        #x = x.permute(0, 2, 1)
        #print(x.shape)
        #x = self.q(x)
        x = self.q(x)

        x = self.decoder(x)
        #print(x.shape)
        x = x.reshape(20,1, 58,58)
        x = x[..., :-self.padding, :-self.padding]
        x = x.permute(0, 2, 3, 1)
        #print('output shape',x.shape)

        return x



    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)
################################################################
# configs
################################################################
TRAIN_PATH = '/content/drive/My Drive/darcy flow/piececonst_r241_N1024_smooth1.mat'
TEST_PATH = '/content/drive/My Drive/darcy flow/piececonst_r241_N1024_smooth2.mat'

ntrain = 1000
ntest = 100

batch_size = 20
learning_rate = 0.001
epochs = 500
iterations = epochs*(ntrain//batch_size)

modes = 12
width = 32

r = 5
h = int(((421 - 1)/r) + 1)
#s = h
s = 49
################################################################
# load data and data normalization
################################################################
reader = MatReader(TRAIN_PATH)
x_train = reader.read_field('coeff')[:ntrain,::r,::r][:,:s,:s]
y_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s]

reader.load_file(TEST_PATH)
x_test = reader.read_field('coeff')[:ntest,::r,::r][:,:s,:s]
y_test = reader.read_field('sol')[:ntest,::r,::r][:,:s,:s]

x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
x_test = x_test.reshape(ntest,s,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

################################################################
# training and evaluation
################################################################
model = FNO2d(modes, modes, width).cuda()
print(count_params(model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=iterations)

myloss = LpLoss(size_average=False)
y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x)
        out = out.reshape(batch_size, s, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        loss.backward()

        optimizer.step()
        scheduler.step()
        train_l2 += loss.item()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x).reshape(batch_size, s, s)
            out = y_normalizer.decode(out)

            test_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

    train_l2/= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_l2, test_l2)

12553253
0 0.9328294330007338 0.20301991987228393 0.1443649172782898
1 0.8685561170004803 0.12031366074085235 0.10231763005256653
2 0.8701802949999546 0.09837596988677978 0.0929461121559143
3 0.873389887000485 0.08653282880783081 0.08437460780143738
4 0.8779977449994476 0.08276524615287781 0.08078545928001404
5 0.8735251289999724 0.08094062626361848 0.0793430733680725
6 0.8895583490002537 0.07841243076324463 0.07797378659248352
7 0.8700911289997748 0.07656078147888183 0.07646454572677612
8 0.8815987859998131 0.07647844004631042 0.07673680067062377
9 0.8751239220000571 0.07461584365367889 0.07493314862251282
10 0.984361804999935 0.07231319200992584 0.07235873460769654
11 1.0255422529999123 0.07229873096942901 0.07220438480377198
12 1.0627977559997817 0.07196300351619721 0.07136666297912597
13 1.0461886459997913 0.07109346091747284 0.07227001070976258
14 0.9854881210003441 0.07042527449131011 0.07255585670471192
15 0.8823119139997289 0.0693593271970749 0.07066143274307252
16 0.8778224259

In [ ]:
# Author： Zephyr Hou
# Time: 2021-08-16

import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

# Parameter Setting
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

train_data =
dataLoader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


class autoEncoder(nn.Module):
    def __init__(self):
        super(autoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(49 * 49, 256),
            nn.Linear(256,128)
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoEncoder().cuda()    # autoEncoder model
loss_func = nn.MSELoss()        # loss function
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataLoader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = loss_func(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

        pic1 = to_img(img.data)
        save_image(pic1, './mlp_img/Ori_image_{}.png'.format(epoch))

